In [8]:
%%time

import pandas as pd 
import scanpy as sc
import phenograph
import parc
from utag import utag
import os 
import numpy as np
import time
import anndata as ad
import scanpy.external as sce
import igraph
#import rapids_singlecell as rsc
os.chdir("/home/abombin/Mawa/multiplex-analysis-web-apps")

CPU times: user 163 µs, sys: 187 µs, total: 350 µs
Wall time: 365 µs


In [ ]:
%%time
adata = sc.read("input/clust_dat.h5ad")
print(adata)


In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, metric='euclidean', n_pcs=None, random_state=42)
sc.tl.leiden(adata, resolution=1, random_state=42, n_iterations=-1, flavor="igraph")
print(adata.obsp['connectivities'])
print(adata.uns['neighbors'])

neighb = adata.uns['neighbors'][0]

In [ ]:
communities, graph, Q = phenograph.cluster(adata.X, clustering_algo="leiden", k=10, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="kdtree",
                                               seed=42, n_iterations=-1, n_jobs=8)

print(graph)

cur_graph  = graph['edges']
cur_graph

In [ ]:
print(type(graph))
print(type(adata.obsp['connectivities']))

In [ ]:
adata = sc.read("input/clust_dat.h5ad")
sc.tl.pca(adata)
sce.tl.phenograph(adata, clustering_algo="leiden", k=30)
print(adata)


In [ ]:
print(adata.obsp)
#print(adata.uns['neighbors'])

In [5]:
%%time

adata = sc.read("input/clust_dat.h5ad")
# Assuming adata is your AnnData object and it has been preprocessed


CPU times: user 97.6 ms, sys: 54.3 ms, total: 152 ms
Wall time: 150 ms


/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [6]:
%%time

communities, graph, Q = phenograph.cluster(adata.X, clustering_algo="leiden", k=10, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="kdtree",
                                               seed=42, n_iterations=-1, n_jobs=8)

adata.obsp['connectivities'] = graph
adata.uns['neighbors'] = {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'n_neighbors': 10, 'method': 'phenograph', 'random_state': 42, 'metric': 'euclidean'}}

Finding 10 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 1.3123178482055664 seconds
Jaccard graph constructed in 1.5891451835632324 seconds
Running Leiden optimization
Leiden completed in 10.359914302825928 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 13.80940556526184 seconds
CPU times: user 21.3 s, sys: 571 ms, total: 21.9 s
Wall time: 13.8 s


In [7]:
%%time
sc.tl.umap(adata)

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/umap/spectral.py:485: RuntimeWarning: divide by zero encountered in divide
  1.0 / sqrt_deg, 0, graph.shape[0], graph.shape[0]


CPU times: user 1min 32s, sys: 2min 14s, total: 3min 46s
Wall time: 18 s


In [9]:
# test with more than a million cells 
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta
adata.layers["counts"] = adata.X.copy()

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [10]:
%%time

communities, graph, Q = phenograph.cluster(adata.X, clustering_algo="leiden", k=10, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="kdtree",
                                               seed=42, n_iterations=-1, n_jobs=8)

adata.obsp['connectivities'] = graph
adata.uns['neighbors'] = {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'n_neighbors': 10, 'method': 'phenograph', 'random_state': 42, 'metric': 'euclidean'}}

Finding 10 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 981.3025534152985 seconds
Jaccard graph constructed in 23.69107675552368 seconds
Running Leiden optimization


In [ ]:
%%time
sc.tl.umap(adata)